## Querying the RAG System with LangChain ##
An LLM is a very capable tool, but its knowledge is limited to the public data it was trained on. It doesn't know about Parasol Company's internal IT procedures or the solutions to our past incidents. How can we make it answer questions using our specific, private data?

There are a few ways to solve this problem:

Full Retraining: This involves re-training the entire model from scratch with our data included. This is incredibly expensive and time-consuming, feasible for only a handful of organizations.

Fine-Tuning: We can "tune" an existing model on our data. This is much faster and cheaper. It's excellent for teaching the model a specific style, tone, or new skill, but less effective for injecting large amounts of factual knowledge. The model must also be re-tuned whenever the data is updated.

Retrieval-Augmented Generation (RAG): This is the technique we will use. We put our private knowledge into an external database (in our case, a Milvus vector database) and "retrieve" the most relevant pieces of information when a user asks a question. We then feed this retrieved context, along with the original question, to the LLM. The LLM uses this specific context to generate a highly relevant and accurate answer. This is powerful because we can continuously update our knowledge base without ever having to retrain the model.

In the previous step, your data science pipeline successfully fetched closed incident tickets from our mock ServiceNow API, generated vector embeddings from them, and loaded them into a Milvus database.

In this notebook, we will use RAG to ask questions about IT support issues and see how the LLM can provide precise answers based on the historical incident data we just ingested.

### Requirements and Imports

If you have selected the right workbench image to launch as per the Lab's instructions, you should already have all the needed libraries. If not uncomment the first line in the next cell to install all the right packages.

In [13]:
# Uncomment the following line only if you have not selected the right workbench image, or are using this notebook outside of the workshop environment.
# !pip install --no-cache-dir --no-dependencies --disable-pip-version-check -r requirements.txt

import json
import transformers
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain_community.llms import VLLMOpenAI
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import Milvus # Use the standard Milvus vector store

# Turn off warnings when downloading the embedding model
transformers.logging.set_verbosity_error()

### Langchain elements

Again, we are going to use Langchain to define our task pipeline.

First, the **LLM** where we will send our queries.

In [ ]:
# LLM Inference Server URL
inference_server_url = "http://granite-3-1-8b-instruct-predictor.shared-llm.svc.cluster.local:8080"

# LLM definition
llm = VLLMOpenAI(           # We are using the vLLM OpenAI-compatible API client. But the Model is running on OpenShift AI, not OpenAI.
    openai_api_key="EMPTY",   # And that is why we don't need an OpenAI key for this.
    openai_api_base= f"{inference_server_url}/v1",
    model_name="granite-3-1-8b-instruct",
    top_p=0.92,
    temperature=0.01,
    max_tokens=512,
    presence_penalty=1.03,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

Then the connection to the **vector database** where we have prepared and stored the California Driver Handbook.

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import Milvus

# 1. Define the embedding model (must match the ingestion pipeline)
print("Loading embedding model...")
embeddings = HuggingFaceEmbeddings(
    model_name="all-MiniLM-L6-v2",
    model_kwargs={'device': 'cpu'},
    show_progress=False,
)
print("Embedding model loaded.")

# 2. Define connection arguments
connection_args = {
    "host": "vectordb-milvus", # The Kubernetes service name for Milvus
    "port": "19530"
}
print(f"Connecting to Milvus at: {connection_args['host']}:{connection_args['port']}")

# 3. Connect to the Milvus vector store, specifying the correct text and vector fields
vector_db = Milvus(
    embedding_function=embeddings,
    connection_args=connection_args,
    collection_name="servicenow_incidents",
    vector_field="embedding",      # Specify the name of your vector field
    text_field="resolution_notes"  # <-- THIS IS THE FIX: Tell LangChain to use this field for page_content
)
print("Successfully connected to Milvus collection 'servicenow_incidents'.")


# 4. Create a retriever to search for relevant documents
retriever = vector_db.as_retriever(search_kwargs={"k": 3})
print("Retriever created successfully.")

We will now define the **template** to use to make our query. Note that this template now contains a **servicenow_incidents** section. That's were the documents returned from the vector database will be injected.

We are now ready to query the model!



In [ ]:
# The 'llm' and 'retriever' objects should already be defined from the previous cells.

# 1. Define a prompt template suitable for a general Q&A over the retrieved documents.
#    This template tells the LLM how to use the context from Milvus to answer the question.
prompt_template_str = """
<|system|>
You are a helpful, respectful and honest assistant named "Parasol Assistant".
You will be given context from past incident tickets and a question.
Your answer should be based only on the provided context.
If the context does not contain the answer, say that you don't have enough information.

Context:
{context}

Question:
{question}
<|assistant|>
"""

prompt = PromptTemplate(
    template=prompt_template_str, input_variables=["context", "question"]
)

# 2. Create the RAG chain with the new prompt.
rag_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    chain_type_kwargs={"prompt": prompt},
    return_source_documents=True,
)

# 3. Define the query and invoke the chain.
query = "Give me information on INC001004"
print(f"Executing RAG chain with query: '{query}'")

resp = rag_chain.invoke({"query": query})

# 4. Print the results.
print("\n--- Answer from LLM ---\n")
print(resp["result"])
print("\n--- Sources Retrieved from Milvus ---\n")
for doc in resp["source_documents"]:
    print(f"  - Incident: {doc.metadata.get('incident_pk', 'N/A')}")
    print(f"    Description: {doc.metadata.get('short_description', 'N/A')}")
    print("-" * 20)

That's it! We now know how to complement our LLM with some external knowledge!

You have successfully built a complete Retrieval-Augmented Generation (RAG) pipeline. By connecting our application to a Milvus vector database populated with our own private data, we've enabled an LLM to answer questions with specific, factual context that it was never trained on. For Parasol Company, this means a support engineer can now ask complex questions and receive answers based on the collective knowledge from thousands of past incident tickets. This is the foundation of a powerful system that can reduce resolution times, improve support consistency, and accelerate new hire training. While this is a huge step forward, the next logical step is to explore how we can not only provide the model with knowledge, but also refine its tone, style, and reasoning capabilities, which we will discuss in the final section.